# dEdx weights calculation

## Input

This notebook expects to find a txt file called "VolumesZPosition.txt". This file is coming from the _SimTracker/TrackerMaterialAnalysis_ package. Just for the record, in the old days we used to correct for the Materials with spaces in their name: 

sed -i -e 's/M_NEMA\ FR4\ plate/M_NEMA_FR4_plate/g' VolumesZPosition.txt

but ok now we use HGC_G10-FR4.

In [ ]:
scenario = "D88"

In [ ]:
inputfile = "VolumesZPosition_%s.txt" % (scenario)

## Some imports

In [ ]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pylab as plt
import seaborn as sns

## Materials Properties

### Materials radiation lengths from Chris, Geant (in mm)

In [ ]:
#In mm
#-------- 
FromChrisMatXo = OrderedDict()
FromChrisMatXo['Copper'] = 14.36
FromChrisMatXo['H_Scintillator'] = 425.4
FromChrisMatXo['HGC_Cables'] = 0.
#FromChrisMatXo['M_NEMA_FR4_plate'] = 175.
#Just duplicate the above in this case
FromChrisMatXo['HGC_G10-FR4'] = 175.
FromChrisMatXo['Silicon'] = 93.66
FromChrisMatXo['Other'] = 0.
FromChrisMatXo['Air'] = 300000.
FromChrisMatXo['StainlessSteel'] = 17.35
FromChrisMatXo['WCu'] = 5.122
FromChrisMatXo['Lead'] = 5.612
#-------- 
FromGeantMatXo = OrderedDict()
FromGeantMatXo['Copper'] = 14.3559
FromGeantMatXo['H_Scintillator'] = 425.393
FromGeantMatXo['HGC_Cables'] = 66.722
#FromGeantMatXo['M_NEMA_FR4_plate'] = 175.056
FromGeantMatXo['HGC_G10-FR4'] = 175.056
FromGeantMatXo['Silicon'] = 93.6762
FromGeantMatXo['Other'] = 0.
FromGeantMatXo['Air'] = 301522.
FromGeantMatXo['StainlessSteel'] = 17.3555
FromGeantMatXo['WCu'] = 5.1225
FromGeantMatXo['Lead'] = 5.6118
#-------- 
df = pd.DataFrame.from_dict([FromChrisMatXo, FromGeantMatXo])
df = df.transpose()
df.columns = ['FromChris', 'FromGeant']
print( '\033[1m' + '            Radiation Lengths in mm')
df

### Materials nuclear interaction lengths from Chris, Geant (in mm)

In [ ]:
#In mm
#-------- 
MatNucIntLength = OrderedDict()
FromChrisMatNucIntLength = OrderedDict()
FromChrisMatNucIntLength['Copper'] = 155.1
FromChrisMatNucIntLength['H_Scintillator'] = 701.3
FromChrisMatNucIntLength['HGC_Cables'] = 0.
#FromChrisMatNucIntLength['M_NEMA_FR4_plate'] = 484.2
#Just duplicate the above in this case
FromChrisMatNucIntLength['HGC_G10-FR4'] = 484.2
FromChrisMatNucIntLength['Silicon'] = 457.5
FromChrisMatNucIntLength['Other'] = 0.
FromChrisMatNucIntLength['Air'] = 700000
FromChrisMatNucIntLength['StainlessSteel'] = 166
FromChrisMatNucIntLength['WCu'] = 119.9
FromChrisMatNucIntLength['Lead'] = 182.6
#-------- 
FromGeantMatNucIntLength = OrderedDict()
FromGeantMatNucIntLength['Copper'] = 155.88
FromGeantMatNucIntLength['H_Scintillator'] = 700.034
FromGeantMatNucIntLength['HGC_Cables'] = 393.71
#FromGeantMatNucIntLength['M_NEMA_FR4_plate'] = 483.429
FromGeantMatNucIntLength['HGC_G10-FR4'] = 483.429
FromGeantMatNucIntLength['Silicon'] = 456.628
FromGeantMatNucIntLength['Other'] = 0.
FromGeantMatNucIntLength['Air'] = 704083
FromGeantMatNucIntLength['StainlessSteel'] = 166.272
FromGeantMatNucIntLength['WCu'] = 120.105
FromGeantMatNucIntLength['Lead'] = 182.472
#-------- 
df = pd.DataFrame.from_dict([FromChrisMatNucIntLength, FromGeantMatNucIntLength])
df = df.transpose()
df.columns = ['FromChris', 'FromGeant']
print( '\033[1m' + '            Nuclear interaction Lengths in mm')
df

### Materials dEdx from PDG, Chris, Geant (in MeV/mm)

In [ ]:
#First in MeV gr^-1 cm^2 and at the end will convert to MeV/mm
dEdx = OrderedDict()
#In gr/cm^3 
rho = OrderedDict()
#--------
#Some elements necessary to build our materials
#Note: Whenever in PDG values (2018) are given in more than one state (e.g gas, liquid)
#we go to the materials.xml file and choose the corresponding value. 
dEdx['Fe'] = 1.451
dEdx['Mn'] = 1.428
dEdx['Cr'] = 1.456
dEdx['Ni'] = 1.468
dEdx['C']  = 1.745
dEdx['0']  = 1.801 
dEdx['H']  = 4.034
dEdx['Br'] = 1.380
dEdx['W']  = 1.145
dEdx['N']  = 1.813
dEdx['Al'] = 1.615


#-------- 
rho['Fe'] = 7.874 
rho['Mn'] = 7.440 
rho['Cr'] = 7.180
rho['Ni'] = 8.902 
rho['C']  = 2.265 
rho['0']  = 1.43**-3 #Here we choose the materials.xml value.
rho['H']  = 0.07080
rho['Br'] = 3.103 
rho['W']  = 19.30 
rho['N']  = 0.8070  
rho['Ar'] = 1.639**-3 #Here we choose the materials.xml value. 
rho['Al'] = 2.7

#--------
#Detector materials (From PDG and Geometry/CMSCommonData/data/materials.xml)
#You need a measurement of the density for a compound which we take from the same file
dEdx['Epoxy'] = 0.53539691*dEdx['C'] + 0.13179314*dEdx['H'] + 0.33280996*dEdx['0']
rho['Epoxy'] = 1.3

dEdx['Kapton'] = 0.59985105*dEdx['C'] + 0.080541353*dEdx['H'] + 0.31960759*dEdx['0']
rho['Kapton'] = 1.11

dEdx['Copper'] = 1.403
rho['Copper'] = 8.960

dEdx['H_Scintillator'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
rho['H_Scintillator'] = 1.032

dEdx['Silicon'] = 1.664
rho['Silicon'] = 2.329

#dEdx['M_NEMA_FR4_plate'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
#rho['M_NEMA_FR4_plate'] = 1.025
dEdx['HGC_G10-FR4'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
rho['HGC_G10-FR4'] = 1.70


dEdx['Other'] = 0.
rho['Other'] = 0.

#dEdx['Air'] = 0.7494*dEdx['N'] + 0.2369*dEdx['0'] + 0.0129*dEdx['Ar'] + 0.0008*dEdx['H']
dEdx['Air'] = 0.
rho['Air'] = 1.214**-3

dEdx['StainlessSteel'] = 0.6996*dEdx['Fe']+0.01*dEdx['Mn']+0.19*dEdx['Cr']+0.1*dEdx['Ni']+0.0004*dEdx['C'];
rho['StainlessSteel'] = 8.02

dEdx['WCu'] = 0.75*dEdx['W']+0.25*dEdx['Copper']
rho['WCu'] = 14.979

dEdx['Lead'] = 1.122 #Pb
rho['Lead'] = 11.35  #Pb

dEdx['HGC_Cables'] = 0.586*dEdx['Copper'] + 0.259*dEdx['C'] + 0.138*dEdx['0'] + 0.017*dEdx['H']
rho['HGC_Cables'] = 2.68

#Two new elements but at this moment they are Air
dEdx['HGC_EEConnector'] = 0.
rho['HGC_EEConnector'] = 1.214**-3

dEdx['HGC_HEConnector'] = 0.
rho['HGC_HEConnector'] = 1.214**-3

dEdx['Polystyrene'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
rho['Polystyrene'] = 1.032

#--------
# Now to the calculation. First, we will loop through the composite materials, not 
# to mess with the elements. 
composite_materials = ['HGC_Cables','Epoxy','Kapton','StainlessSteel','M_NEMA_FR4_plate','H_Scintillator', 'Air', 'WCu','HGC_EEConnector','HGC_HEConnector','Polystyrene']
for element in dEdx: 
    if element not in composite_materials: continue
    dEdx[element] = (rho[element] * dEdx[element]) / 10.
#And for the rest
for element in dEdx: 
    if element in composite_materials: continue
    dEdx[element] = (rho[element] * dEdx[element]) / 10.


#--------
FromChrisdEdx = OrderedDict()
FromChrisdEdx['Copper'] = 1.26
FromChrisdEdx['H_Scintillator'] = 0.395
FromChrisdEdx['Epoxy'] = 0.
FromChrisdEdx['Kapton'] = 0.
FromChrisdEdx['HGC_Cables'] = 0.
FromChrisdEdx['Al'] = 0.
#FromChrisdEdx['M_NEMA_FR4_plate'] = 0.322
#Just duplicate the above
FromChrisdEdx['HGC_G10-FR4'] = 0.322
FromChrisdEdx['Silicon'] = 0.388
FromChrisdEdx['Other'] = 0.
FromChrisdEdx['Air'] = 0.
FromChrisdEdx['StainlessSteel'] = 1.14
FromChrisdEdx['WCu'] = 1.81
FromChrisdEdx['Lead'] = 1.27
#-------- 
#The range below is from commands like: 
# Always recheck columns in txt, so that array is pointing to desired column. 
#array=($(cat VolumesZPosition.txt | grep Copper | awk '{print $7}'))
#IFS=$'\n'
#Highest value
#echo "${array[*]}" | sort -nr | head -n1
#Lowest value
#echo "${array[*]}" | sort -nr | tail -n1

FromGeantdEdxWithGetDEDX = OrderedDict()
FromGeantdEdxWithGetDEDX['Copper'] = "1.14494 - 1.19191"
FromGeantdEdxWithGetDEDX['H_Scintillator'] = "0.155602 - 0.156968"
FromGeantdEdxWithGetDEDX['HGC_Cables'] = "0.334163 - 0.343928"
#FromGeantdEdxWithGetDEDX['M_NEMA_FR4_plate'] = "0.200079 - 0.203959"
FromGeantdEdxWithGetDEDX['Epoxy'] = 0.
FromGeantdEdxWithGetDEDX['Kapton'] = 0.
FromGeantdEdxWithGetDEDX['Al'] = 0.
FromGeantdEdxWithGetDEDX['HGC_G10-FR4'] = "0.200079 - 0.203959"
FromGeantdEdxWithGetDEDX['Silicon'] = "0.298729 - 0.349232"
FromGeantdEdxWithGetDEDX['Other'] = 0.
FromGeantdEdxWithGetDEDX['Air'] = "0.000162487 - 0.000200878"
FromGeantdEdxWithGetDEDX['StainlessSteel'] = "1.17868 - 1.24005"
FromGeantdEdxWithGetDEDX['WCu'] = "1.4676 - 1.54044"
FromGeantdEdxWithGetDEDX['Lead'] = "1.28276 - 1.39512"
#-------- 
#The range below is from commands like: 
# Always recheck columns in txt, so that array is pointing to desired column. 
#array=($(cat VolumesZPosition.txt | grep Copper | awk '{print $9}'))
#IFS=$'\n'
#Highest value
#echo "${array[*]}" | sort -nr | head -n1
#Lowest value
#echo "${array[*]}" | sort -nr | tail -n1
FromGeantdEdxWithComputeTotalDEDX = OrderedDict()
FromGeantdEdxWithComputeTotalDEDX['Copper'] = "1.25937 - 1.51496"
FromGeantdEdxWithComputeTotalDEDX['H_Scintillator'] = "0.212843 - 0.230636"
FromGeantdEdxWithComputeTotalDEDX['HGC_Cables'] = "0.458855 - 0.507352"
#FromGeantdEdxWithComputeTotalDEDX['M_NEMA_FR4_plate'] = "0.319344 - 0.364818"
FromGeantdEdxWithComputeTotalDEDX['Epoxy'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Kapton'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Al'] = 0.
FromGeantdEdxWithComputeTotalDEDX['HGC_G10-FR4'] = "0.319344 - 0.364818"
FromGeantdEdxWithComputeTotalDEDX['Silicon'] = "0.38849 - 0.442262"
FromGeantdEdxWithComputeTotalDEDX['Other'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Air'] = "0.000221838 - 0.000303024"
FromGeantdEdxWithComputeTotalDEDX['StainlessSteel'] = "1.18145 - 1.40497"
FromGeantdEdxWithComputeTotalDEDX['WCu'] = "1.85741 - 2.18222"
FromGeantdEdxWithComputeTotalDEDX['Lead'] = "1.28388 - 1.52941"
#-------- 
df = pd.DataFrame.from_dict([dEdx,FromChrisdEdx, FromGeantdEdxWithGetDEDX, FromGeantdEdxWithComputeTotalDEDX])
df = df.transpose()[10:]
df.columns = ['FromPDG','FromChris', 'FromGeantWithGetDEDX', 'FromGeantWithComputeTotalDEDX']
print( '\033[1m' + '                              dEdx in MeV/mm')
df

### Choice for radiation lengths values -> Geant values

In [ ]:
# Current choice -> Geant values
#In mm
MatXo = OrderedDict()
MatXo['Copper'] = 14.3559
MatXo['H_Scintillator'] = 425.393
MatXo['HGC_Cables'] = 66.722
MatXo['Epoxy'] = 315.901
MatXo['Kapton'] = 365.309
#MatXo['M_NEMA_FR4_plate'] = 175.056
MatXo['HGC_G10-FR4'] = 175.056
MatXo['Silicon'] = 93.6762
MatXo['Other'] = 0.
MatXo['Air'] = 301522.
MatXo['StainlessSteel'] = 17.3555
MatXo['WCu'] = 5.1225
MatXo['Lead'] = 5.6118
#Definitions below is the same as with Air at the moment
MatXo['HGC_EEConnector'] = 301522.
MatXo['HGC_HEConnector'] = 301522.
#Polystyrene definition same as H_Scintillator at the moment 
MatXo['Polystyrene'] = 425.393

In [ ]:
# Let's read the input file
# Mat is prepoint material, Z is post point material start, so 
# upper edge of prepoint material. Etable,Efull is the energy loss in the 
# prepoint volume with GetDEDX and ComputeTotalDEDX.  
matZ = pd.read_csv(inputfile, sep=" ", header=None, names=["Mat", "Z", "Eta", "R", "Etable", "Efull"], index_col=False)

In [ ]:
# We will make a new column with the physical thickness of the volumes in mm
matZ["PhysThickInmm"] = abs(matZ["Z"].shift(-1) -  matZ["Z"])
matZ["PhysThickInmm"] = matZ["PhysThickInmm"].shift(1)

In [ ]:
# We will add a column that indicates the track the relevant volume belongs to. 
# The logic is that right before the next track "PhysThickInmm" column will be 
# very large. 
matZ["trackflag"] = matZ.apply(lambda row: True if row["PhysThickInmm"] < 2000. else False ,axis=1)
matZ["tracknum"] = (( matZ["trackflag"] == False) & (matZ["trackflag"].shift() == True)).cumsum()
matZ["tracknum"] = matZ["tracknum"].shift(1)
matZ = matZ.drop('trackflag', 1)

In [ ]:
# Now that we have the tracknum we will not let the last Copper volume to 
# its huge thickness due to track change. According to TDR BH back is at 5137.7 mm 
# so we put 6.0 mm for that Copper. In any case this do *not* effect the 
# dEdx weights calculation since it is after the sensitive material of the last layer. 
# UPDATE: Seems that now last volume is SS. No effect but in any case we put 35/66 mm -> 35
matZ.loc[ matZ["PhysThickInmm"] > 2000. , "PhysThickInmm" ] = 35.0

In [ ]:
# The line below is when comparing with Chris which has no HGC_Cables. 
# matZ = matZ.query('Mat != "HGC_Cables"')

# Again, adding a new column with the physical thickness of the volumes in radiation lengths
matZ["PhysThickInXo"] = matZ.apply(lambda row: row["PhysThickInmm"] / MatXo[row["Mat"]],axis=1)

In [ ]:
# Adding a new column with the dEdx of the material that the volumes is build
matZ["dEdx"] = matZ.apply(lambda row: dEdx[row["Mat"]],axis=1)

In [ ]:
# Another column with the dEdx times thickness to help us with the calculation of the 
# final dEdx weights 
matZ["dEdxtimesdx"] = matZ["dEdx"] * matZ["PhysThickInmm"]

In [ ]:
# And here a column with the cumulative sum
matZ["dEdxtimesdxCum"] = matZ.groupby('tracknum')["dEdxtimesdx"].cumsum()

In [ ]:
# And here a column with the cumulative sum for Etable
matZ["EtableCum"] = matZ.groupby('tracknum')["Etable"].cumsum()

In [ ]:
# And here a column with the cumulative sum for Efull
matZ["EfullCum"] = matZ.groupby('tracknum')["Efull"].cumsum()

In [ ]:
# And here a column for the cumulative sum in radiation length 
matZ["PhysThickInXoCum"] = matZ.groupby('tracknum')["PhysThickInXo"].cumsum()

In [ ]:
# We will add a column that indicates the layer the relevant volume belongs to. 
# The logic is that if the previous material is Silicon or Scintillator
# we change layer.
matZ["layerflag"] = matZ.apply(lambda row: False if row["Mat"] == "Silicon" or row["Mat"] == "H_Scintillator" else True ,axis=1)
matZ["layer"] = ( matZ["layerflag"] == True) & (matZ["layerflag"].shift(1) == False) 
matZ["layer"] = matZ.groupby('tracknum')["layer"].cumsum()
#The convention is that layers starts from 1
matZ["layer"] = matZ.apply(lambda row: row["layer"] + 1,axis=1)

In [ ]:
# Drop auxillary columns
#We need layerflag for not counting the silicon/scintillator in the dedx. 
#matZ = matZ.drop('layerflag', 1)

In [ ]:
# This was my misunderstanding. There should be 53 in the layers column since 
# after the last scintillator or silicon the index will increase, there is 
# material there. In case we want to filter them out uncomment the following. 
# Be careful! Filter chooses and not disgards!
# matZ = matZ.groupby('tracknum').filter(lambda g: ~(g['layer'] == 53.0).any()  ) 

In [ ]:
#display(matZ)
#matZ[(matZ["layer"] == 1) & matZ["tracknum"] == 20]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#matZ[(matZ["tracknum"] == 6.0)]
matZ = matZ.dropna()
#matZ[ (matZ['tracknum'] == 6.0) &  (matZ["layer"] == 46) ]
#matZ[(matZ["Mat"] == "Kapton") & (matZ["tracknum"] == 2.0)]
#matZ[(matZ["Mat"] == "HGC_Cables")]
#matZ[(matZ["Mat"] == "HGC_Cables") & (matZ["layer"] == 43)]
#matZ[(matZ["Mat"] == "Air") & (abs(matZ["eta"]) > 1.5) & (abs(matZ["eta"]) < 2.0)]
#matZ[ (matZ["Z"] > 3650) & (matZ["Z"] < 3700) & (matZ["Mat"] == "Silicon") & (matZ["R"] < 900)]
#matZ[ (matZ["Z"] > 3594) & (matZ["Z"] < 3700) & (matZ["tracknum"] == 4)]
#matZ[ (matZ["Mat"] == "Copper") & (matZ["Z"] > 3600) & (matZ["Z"] < 3750) & (matZ["tracknum"] == 4) ]
#matZ[ (matZ["Mat"] == "Copper") & (matZ["Z"] > 4000) & (matZ["Z"] < 4750) & (matZ["tracknum"] == 4) ]
#matZ[ (matZ["Z"] > 4000) & (matZ["Z"] < 4750) & (matZ["tracknum"] == 7) ]
#matZ[ (matZ["Mat"] == "Lead") & (matZ["tracknum"] == 4) ]
#matZ[ (matZ["Mat"] == "H_Scintillator")  ]
#matZ[ (matZ["Mat"] == "HGC_Cables")  ]
#matZ[ (matZ["Z"] > 3210) & (matZ["Z"] < 3350) & (matZ["tracknum"] == 4) ]
#matZ[ (matZ["Z"] > 3650) & (matZ["Z"] < 3700) & (matZ["tracknum"] == 4) ]
#matZ[ (matZ["Z"] > 4750) & (matZ["Z"] < 5250) & (matZ["tracknum"] == 18) ]
#matZ[ (matZ["Mat"] == "Copper") & (matZ["Z"] > 3210) & (matZ["Z"] < 3650) & (matZ["tracknum"] == 4) & (matZ["PhysThickInmm"] > 5. )]
#matZ[ (matZ["Z"] > 4550) & (matZ["Z"] < 4650)& (matZ["tracknum"] == 18)]
#matZ[ (matZ["Z"] > 4290) & (matZ["Z"] < 4350) & (matZ["Mat"] == "Epoxy")]         
#matZ[ (matZ["Z"] > 4290) & (matZ["Z"] < 4650) & (matZ["Mat"] == "H_Scintillator")]
#matZ[ (matZ["Z"] > 4290) & (matZ["Z"] < 4650) & (matZ["Mat"] == "Silicon")]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["tracknum"] == 6) ]
#matZ[ (matZ["Mat"] == "H_Scintillator") & (matZ["tracknum"] == 18) ]
#matZ[ (matZ["Mat"] == "HGC_G10-FR4") & (matZ["tracknum"] == 4) ]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["Z"] < 3900) & (matZ["Z"] > 0) & (matZ["PhysThickInmm"] > 0.22)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["layer"] > 28) & (matZ["layer"] < 36)& (matZ["PhysThickInmm"] < 0.22)]
#matZ[matZ["layer"] >= 0]
#matZ[(matZ["Mat"] == "WCu") & (matZ["Z"] > 3950) & (matZ["Z"] < 4000)]

In [ ]:
# We will make a new column with the cassete space in mm
outforSS = matZ[ (matZ["Mat"] == 'StainlessSteel') & (matZ["PhysThickInmm"] != 2.5) & (matZ["tracknum"] == 4) ]
outforSS
abs(outforSS["Z"].shift(-1) -  outforSS["Z"]).shift(1)
#outforSS["CasseteSpace"] = abs(outforSS["Z"].shift(-1) -  outforSS["Z"])
#outforSS["PhysThickInmm"] = outforSS["PhysThickInmm"].shift(1)

In [ ]:
#I will save here the thicknesses to be able to see the differences in files
for themat in ['Copper','Polystyrene','HGC_EEConnector','HGC_HEConnector','H_Scintillator','HGC_Cables','Epoxy','Kapton','HGC_G10-FR4','Silicon','Other','Air','StainlessSteel','WCu','Lead']:
    outtofile = matZ[ (matZ["Mat"] == themat) & (matZ["tracknum"] == 7) ]
    outtofile[["Mat","Z","PhysThickInmm","tracknum","layer"]].round(3).to_csv(r'/eos/user/a/apsallid/www/material_%s_%s.txt' %(themat,scenario), index=None, sep=' ', mode='w')

#matZ[matZ["Z"]>0]
#matZ[matZ["PhysThickInmm"]> 2000.]
#matZ[matZ["tracknum"] == 2 ]
#matZ[ (matZ["tracknum"] == 20) & (matZ["layer"] > 10) ]
#matZ[450:500]
#For the cumulative material in front of layers. 
newmatZ = matZ[ (matZ['tracknum'] == 6.0) & (matZ['layerflag'] == True) & (matZ['layerflag'].shift(-1) == False)]
newmatZ[['layer', 'PhysThickInXoCum']]
newmatZ[['layer', 'PhysThickInXoCum']].round(3).to_csv(r'/eos/user/a/apsallid/www/%s.cumulative.xo'%(scenario), header=None, index=None, sep=' ', mode='w')

In [ ]:
# The dEdx weights calculation doesn't include the sensitive material. 
matZ = matZ.drop(matZ[ (matZ.Mat == "Silicon") | (matZ.Mat == "H_Scintillator") ].index)

In [ ]:
# We will add a column with the dedx weights. First dedxtimesdx sum per layer 
# This is with the theoretical dedx
mdEdxtimesdxsumperlayer = matZ.groupby(['tracknum','layer'])["dEdxtimesdx"].sum()
# And now with the detailed simulation
mdEdxtimesdxsumperlayer_detailedtable = matZ.groupby(['tracknum','layer'])["Etable"].sum()
mdEdxtimesdxsumperlayer_detailedfull = matZ.groupby(['tracknum','layer'])["Efull"].sum()

#type(mdEdxtimesdxsumperlayer)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.to_frame().reset_index()

#mdEdxtimesdxsumperlayer
#type(mdEdxtimesdxsumperlayer)

In [ ]:
#Let's put also the accumulated energy loss
mdEdxtimesdxsumperlayer["dEdxtimesdxCum"] = mdEdxtimesdxsumperlayer.groupby('tracknum')["dEdxtimesdx"].cumsum()
mdEdxtimesdxsumperlayer_detailedtable["EtableCum"] = mdEdxtimesdxsumperlayer_detailedtable.groupby('tracknum')["Etable"].cumsum()
mdEdxtimesdxsumperlayer_detailedfull["EfullCum"] = mdEdxtimesdxsumperlayer_detailedfull.groupby('tracknum')["Efull"].cumsum()

In [ ]:
#Final weights
mdEdxtimesdxsumperlayer["dedxweights"] = (mdEdxtimesdxsumperlayer["dEdxtimesdx"] + mdEdxtimesdxsumperlayer["dEdxtimesdx"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedtable["dedxweights_detailedsimulationtable"] = (mdEdxtimesdxsumperlayer_detailedtable["Etable"] + mdEdxtimesdxsumperlayer_detailedtable["Etable"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedfull["dedxweights_detailedsimulationfull"] = (mdEdxtimesdxsumperlayer_detailedfull["Efull"] + mdEdxtimesdxsumperlayer_detailedfull["Efull"].shift(-1))/2

In [ ]:
#Hack for the last layer
mdEdxtimesdxsumperlayer.loc[mdEdxtimesdxsumperlayer["layer"] == 47.0, "dedxweights"] = 83.328143
mdEdxtimesdxsumperlayer_detailedtable.loc[mdEdxtimesdxsumperlayer_detailedtable["layer"] == 52.0, "dedxweights_detailedsimulationtable"] = 91.800143
mdEdxtimesdxsumperlayer_detailedfull.loc[mdEdxtimesdxsumperlayer_detailedfull["layer"] == 52.0, "dedxweights_detailedsimulationfull"] = 98.971647

#Drop duplicates not needed columns
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.drop(['tracknum', 'layer'], axis=1)
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.drop(['tracknum', 'layer'], axis=1)

#mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ]
#mdEdxtimesdxsumperlayer_detailed[ mdEdxtimesdxsumperlayer_detailed['tracknum'] == 6.0  ]
mdEdxtimesdxsumperlayer = pd.concat([mdEdxtimesdxsumperlayer, mdEdxtimesdxsumperlayer_detailedtable, mdEdxtimesdxsumperlayer_detailedfull], axis=1)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.dropna()

#Adding two columns dedxtable/dedxtheory and dedxfull/dedxtheory
mdEdxtimesdxsumperlayer["EtableoverEtheory"] = mdEdxtimesdxsumperlayer["Etable"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer["EfulloverEtheory"] = mdEdxtimesdxsumperlayer["Efull"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1].dropna()

In [ ]:
mdEdxtimesdxsumperlayer[["layer","EtableoverEtheory","EfulloverEtheory"]][ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1] 

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

mdEdxtimesdxsumperlayer[["layer","dEdxtimesdx","dedxweights","dedxweights_detailedsimulationtable","dedxweights_detailedsimulationfull"]][ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1] 

In [ ]:
forthededxtable = mdEdxtimesdxsumperlayer[["layer","dEdxtimesdx","dEdxtimesdxCum","EtableCum","EfullCum"]][ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1] 

pd.options.display.float_format = '{:,.2f}'.format

mdEdxtimesdxsumperlayer[["layer","dedxweights"]][ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1] 

In [ ]:
forthededxtable[["layer","dEdxtimesdxCum"]]
forthededxtable[["layer","dEdxtimesdxCum"]].round(3).to_csv(r'/eos/user/a/apsallid/www/%s.cumulative.dedx'%(scenario), header=None, index=None, sep=' ', mode='w')
forthededxweights = mdEdxtimesdxsumperlayer[["layer","dedxweights"]][ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1] 
forthededxweights[["layer","dedxweights"]].round(2).to_csv(r'/eos/user/a/apsallid/www/%s.dedx.weights'%(scenario), header=None, index=None, sep=' ', mode='w')
forthededxperlayer = mdEdxtimesdxsumperlayer[["layer","dEdxtimesdx"]][ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ][:-1]
forthededxperlayer[["layer","dEdxtimesdx"]].round(2).to_csv(r'/eos/user/a/apsallid/www/%s.dedx.perlayer'%(scenario), header=None, index=None, sep=' ', mode='w')